In [1]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta

# Add the src directory to the system path and import functions
sys.path.append(os.path.abspath('../src'))
from schedule_weather import generate_activity_list, Scheduler
from weather_constraints import get_daylight_windows, get_tide_windows

from plot_weather import plot_weather

In [2]:
#read in activity data from excel
#update to be relative to your project structure
main_input_folder_path = r"C:\Users\btweb\Downloads\WeatherWindows"
path = os.path.join(main_input_folder_path, "WeatherWindows.xlsx")
act_df = pd.read_excel(path, sheet_name="Activities", skiprows=1)
constraints_df = pd.read_excel(path, sheet_name="Constraints", skiprows=1)

activities = generate_activity_list(act_df, constraints_df)

c:\Users\btweb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
# 1. Define target activity and completion date
target_activity = "Pilot Punch Out"
input_completion_date = datetime(2025, 12, 9, 3, 0)

In [4]:
# 2. Generate daylight and tidal windows around target date ±7 days
window_data_start = input_completion_date - timedelta(days=3)
window_data_end = input_completion_date + timedelta(days=7)

daylight_windows = get_daylight_windows(
    start_date=window_data_start,
    end_date=window_data_end,
    save_csv=None
)

tidal_data_path = os.path.join(main_input_folder_path, "Reference/harwich_tide.csv")
hw_windows, lw_windows, events = get_tide_windows(
    tidal_data_path,
    start_date=window_data_start,
    end_date=window_data_end,
    skiprows=2,
    datetime_col="Date/Time",
    height_col="Height(m)",
    slack_window_after=1.5,
    slack_window_before=1.5,
    plot=False,
    save_csv=None
)

c:\Users\btweb\Documents\Python Scripts\WeatherOpsPlanner\src\weather_constraints.py:56: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[datetime_col] = pd.to_datetime(df[datetime_col], errors='coerce')


Loaded data from 2025-10-01 00:00:00 to 2026-10-31 23:50:00 with 57024 records.
Filtered data from 2025-12-06 03:00:00 to 2025-12-16 03:00:00 with 1441 records.


In [5]:
# 3. Create Scheduler instance with activities and environmental windows
scheduler = Scheduler(
    activities=activities,
    daylight_windows=daylight_windows,   # List of (start, end) daylight datetime pairs
    hw_tide_windows=hw_windows,           # High/slack tide windows as datetime pairs
    lw_tide_windows=lw_windows            # Low tide windows as datetime pairs
)

In [6]:
# 4. Run backward scheduling from target completion date for target activity
scheduled_activities = scheduler.schedule_around_target(target_activity, input_completion_date)



In [7]:
# 5. Export scheduled activities to pandas DataFrame for analysis
schedule_df = scheduler.to_dataframe()

# 6. Calculate total schedule duration: from earliest start to latest end of activities
total_duration = schedule_df['End'].max() - schedule_df['Start'].min()

# 7. Print total schedule duration
print(f"Total duration of the schedule: {total_duration}")

# 8. Display the first few scheduled activities in the DataFrame
schedule_df.head()


Total duration of the schedule: 9 days 19:22:39.657408


,ID,Name,Start,End,Duration,Group
0,PM01,Marine spread readiness,NaT,NaT,5.0,Pipe Management
1,CTV01,Personel transfer prior to pipe pull,NaT,NaT,2.0,Crew Transfer Vessel
2,HDD01,Pilot Punch Out,2025-12-09 00:00:00.000000,2025-12-09 03:00:00.000000,3.0,Horizontal Directional Drilling
3,CTV02,Bathy Survey,2025-12-09 07:50:58.342680,2025-12-09 10:50:58.342680,3.0,Crew Transfer Vessel
4,HDD02,Pilot Pull Back,2025-12-09 03:00:00.000000,2025-12-09 09:00:00.000000,6.0,Horizontal Directional Drilling


In [8]:
# Load the tidal data
file_path = os.path.join(main_input_folder_path,"Reference\harwich_tide.csv")
tide_df = pd.read_csv(file_path, skiprows=3, names=["DateTime", "Height"])

# Convert the 'DateTime' column to datetime format
tide_df['DateTime'] = pd.to_datetime(tide_df['DateTime'], errors='coerce')

# Filter the data to include only rows where the month is December
december_tide_df = tide_df[tide_df['DateTime'].dt.month == 12]

tide_window_df = events


<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:2: SyntaxWarning: invalid escape sequence '\h'
C:\Users\btweb\AppData\Local\Temp\ipykernel_12240\2389328646.py:2: SyntaxWarning: invalid escape sequence '\h'
  file_path = os.path.join(main_input_folder_path,"Reference\harwich_tide.csv")


In [9]:
fig = plot_weather(schedule_df, scheduler.daylight_windows, december_tide_df, tide_window_df)

KeyError: 'Duration (hours)'

In [ ]:
outputpath=os.path.join(main_input_folder_path, "pipe_pull_schedule.html")
fig.write_html(outputpath, include_plotlyjs='cdn')